In [1]:
# Tabular data structure library 

import pandas as pd 

# Linear algebra library of python 

import numpy as np 

# For CUDA compatibility

import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1" # Uncomment this line for use of CPU only otherwise GPU scaling is enabled

# For visualisation and plotting

import matplotlib.pyplot as plt

# Import train/test split

from sklearn.model_selection import train_test_split

# Encoding

from sklearn.metrics import confusion_matrix, classification_report

from sklearn.preprocessing import LabelEncoder

import itertools

from keras.utils import np_utils

from tensorflow import keras

from tensorflow.keras.layers import Conv1D, Dense, Input, Activation, Multiply, Add, ReLU, Softmax, GlobalAveragePooling1D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import HeNormal

# Custom confusion matrix package

import confusion_matrix_pretty_print as cmpp

import pickle

from sklearn import metrics

from pycm import *

In [2]:
# !pip install pycm==3.1

In [3]:
# Load model

model = load_model("saved_models/30_epoch_WaveNet24lyrs_whitenoise.hdf5")

In [4]:
# Load data

# Load data and labels in

# X = np.load('wav_files_np.npy')
# y = np.load('labels_np.npy')

# Load data and labels in #Noise


X = np.load('wav_files_whitenoise.npy',allow_pickle=True)
y = np.load('labels_whitenoise.npy',allow_pickle=True)

# Encode the labels 

encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)

dummy_y = np_utils.to_categorical(encoded_Y)

# Split the train and test data

X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.33, random_state=42,stratify=dummy_y)

In [5]:
# Predict

predictions = np.argmax(model.predict(X_test), axis=-1)

predictions_dummy = model.predict(X_test)

np.unique(predictions) # Unique values for its respective class

Original_y = np.argmax(y_test,axis=1)

score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]} - & - Test accuracy: {score[1]}')

Test loss: 0.4849763512611389 - & - Test accuracy: 0.8600103259086609


In [6]:
cm = ConfusionMatrix(actual_vector=Original_y, predict_vector=predictions) # Create CM From Data
cm.stat(overall_param=["FPR Macro","FPR Micro","FNR Macro","FNR Micro"],class_param=None)

Overall Statistics : 

FNR Macro                                                         0.15156
FNR Micro                                                         0.13999
FPR Macro                                                         0.00412
FPR Micro                                                         0.00412

Class Statistics :

Classes                                                           0             1             2             3             4             5             6             7             8             9             10            11            12            13            14            15            16            17            18            19            20            21            22            23            24            25            26            27            28            29            30            31            32            33            34            
ACC(Accuracy)                                                     0.99382       0.99298       0.99467  

C:\Users\Lieuwe\anaconda3\lib\site-packages\pycm\pycm_obj.py:182: RuntimeWarning: The confusion matrix is a high dimension matrix and won't be demonstrated properly.
If confusion matrix has too many zeros (sparse matrix) you can set `sparse` flag to True in printing functions otherwise by using save_csv method to save the confusion matrix in csv format you'll have better demonstration.
  warn(CLASS_NUMBER_WARNING, RuntimeWarning)


In [7]:
assert False

AssertionError: 

In [ ]:
fnr_all = dict()
fpr_all = dict()
roc_auc = dict()

for i in range(35):
    fpr, tpr, _ = metrics.roc_curve(y_test[:, i], predictions_dummy[:, i])
    fnr = 1 - tpr
    
    # store results
    fnr_all[i] = fnr
    fpr_all[i] = fpr
    
# Compute micro results
micro_fpr, micro_tpr, _ = metrics.roc_curve(y_test.ravel(), predictions_dummy.ravel())
micro_fnr = 1 - micro_tpr
fnr_all['micro'] = micro_fnr
fpr_all['micro'] = micro_fpr
metrics.auc(fpr_all["micro"], fnr_all["micro"])

In [ ]:
print(fpr_all['micro'])
print(fnr_all['micro'])